# Investment Simulation System

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
%pip install pandas_ta_classic

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tqdm 

Note: you may need to restart the kernel to use updated packages.


### Imports

In [4]:
import os
import site

import os
import shutil
import gc
import warnings
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm
from datetime import timedelta

warnings.filterwarnings('ignore')

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

from sklearn.metrics import (
    precision_score, recall_score, f1_score, matthews_corrcoef,
    mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
)


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [7]:
pandas_ta_classic_path = None
for sp in site.getsitepackages():
    pandas_ta_classic_path = os.path.join(sp, 'pandas_ta_classic')
    if os.path.exists(pandas_ta_classic_path):
        break

if pandas_ta_classic_path:
    squeeze_pro_path = os.path.join(pandas_ta_classic_path, 'momentum', 'squeeze_pro.py')
    if os.path.exists(squeeze_pro_path):
        try:
            with open(squeeze_pro_path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            fixed = False
            for line in lines:
                if "from numpy import NaN as npNaN" in line:
                    new_lines.append(line.replace("from numpy import NaN as npNaN", "# from numpy import NaN as npNaN\nimport numpy as np\n"))
                    fixed = True
                    print("Modified import statement in squeeze_pro.py")
                else:
                    new_lines.append(line)

            if fixed:
                with open(squeeze_pro_path, 'w') as f:
                    f.writelines(new_lines)
                print("Successfully patched pandas_ta_classic/momentum/squeeze_pro.py")
            else:
                print("Could not find the problematic import line in squeeze_pro.py")

        except Exception as e:
            print(f"Error modifying squeeze_pro.py: {e}")
    else:
        print(f"Could not find squeeze_pro.py at {squeeze_pro_path}")
else:
    print("Could not find the pandas_ta_classic library installation path.")

import pandas_ta_classic as ta

Could not find the problematic import line in squeeze_pro.py


### Configurations

In [8]:
MODEL_SAVE_PATH = "trained_models/"
MIN_SEQUENCE_LENGTH = 12  # Minimum sequence length for any company
MAX_SEQUENCE_LENGTH = 12  # Maximum sequence length to cap computational cost
INITIAL_TRAINING_DAYS = 1100  # Number of days to use for initial training only
KELLY_FRACTION = 0.1
SECTOR_CONFIDENCE_THRESHOLD = 0.30
RETRAIN_INTERVAL = 200
MAX_DAY_GAP = 5  # Maximum allowed gap in trading days (to account for weekends/holidays)

In [9]:
MAX_TOTAL_UTILIZATION      = 0.70   # never deploy more than 70% of capital at once
MAX_NEW_DAILY_UTILIZATION  = 0.30   # new trades today ≤ 30% of capital
MAX_TICKER_UTILIZATION     = 0.20   # per ticker cap (sum of all overlapping trades)
MAX_SECTOR_UTILIZATION     = 0.45   # per sector cap (sum across its tickers)
MAX_POSITION_SIZE          = 0.1   # single-trade cap

MAX_POSITIONS_PER_TICKER   = 3      # ladder depth per ticker
MIN_TRADE_DOLLARS          = 100.0  # skip dust trades

KELLY_TRADE_CAP            = 0.30   # cap raw Kelly per trade (before other caps)
DRAWDOWN_THROTTLE_LEVELS   = [(0.05, 0.50), (0.10, 0.25)]  # (peak_dd, utilization_multiplier)


In [10]:
import random


def set_global_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_global_seeds(42)

#### Define Horizon Target
For a horizon H, compute both the direction and H-day return off the same base day t

In [11]:
def add_horizon_targets(df: pd.DataFrame, H: int, price_col='close_price') -> pd.DataFrame:
    df = df.sort_values(['ticker','date']).copy()
    df[f'ret_{H}d']    = df.groupby('ticker')[price_col].shift(-H) / df[price_col] - 1.0
    df[f'target_{H}d'] = (df[f'ret_{H}d'] > 0).astype(int)
    return df


#### Identify Contiguous Periods
Groups rows for a ticker into blocks where data gaps never exceed MAX_DAY_GAP

In [12]:
def identify_contiguous_periods(df: pd.DataFrame, max_gap_days: int = MAX_DAY_GAP) -> list:

    if df.empty:
        return []

    df = df.sort_values('date').reset_index(drop=True)
    dates = pd.to_datetime(df['date'])

    contiguous_periods = []
    start_idx = 0

    for i in range(1, len(dates)):
        gap = (dates[i] - dates[i-1]).days
        if gap > max_gap_days:
            # End current period and start new one
            contiguous_periods.append((start_idx, i-1))
            start_idx = i


    contiguous_periods.append((start_idx, len(dates)-1))


    contiguous_periods = [(s, e) for s, e in contiguous_periods if e - s >= MIN_SEQUENCE_LENGTH]

    return contiguous_periods

#### Calculate Dynamic Sequence Length 
Pick a sequence length tailored to the amount of history available for a ticker.
Avoids hard-coding a single window size for short vs long histories.

In [13]:
def calculate_dynamic_sequence_length(company_data_df: pd.DataFrame,
                                     min_length: int = MIN_SEQUENCE_LENGTH,
                                     max_length: int = MAX_SEQUENCE_LENGTH,
                                     target_fraction: float = 0.15) -> int:

    total_days = len(company_data_df)


    dynamic_length = int(total_days * target_fraction)
    dynamic_length = max(min_length, min(dynamic_length, max_length))

    return dynamic_length

#### Create Contiguous Sequences
Turn each contiguous period into overlapping (sequence_length) windows (X) and next-step labels (y).
Produces the actual training/validation tensors for the model.

In [14]:
def create_contiguous_sequences(data: np.ndarray, targets: np.ndarray,
                               contiguous_periods: list, sequence_length: int, H: int):

    X, y = [], []

    for start_idx, end_idx in contiguous_periods:
        usable_end = end_idx - H
        period_length = usable_end - start_idx + 1
        if period_length < sequence_length + 1:
            continue

        for i in range(start_idx + sequence_length -1 , usable_end + 1):
            X.append(data[i-sequence_length + 1 : i +1])
            y.append(targets[i + H])

    return np.array(X) if X else np.array([]), np.array(y) if y else np.array([])

#### Create Target Variable
Build the binary classification target per row.

In [15]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:

    print("Creating target variable...")
    df = df.sort_values(by=['ticker', 'date']).copy()
    df['next_day_close'] = df.groupby('ticker')['close_price'].shift(-1)
    df['target'] = (df['next_day_close'] > df['close_price']).astype(int)
    df.dropna(subset=['next_day_close'], inplace=True)
    df['target'] = df['target'].astype(int)
    print("Target variable created.")
    return df

#### Track Current Exposures from Open Trades
Settle exits first each day, then compute exposure books before opening new ones.

In [16]:
from collections import defaultdict

def compute_exposures(open_positions, sectors_by_ticker):
    expo_ticker = defaultdict(float)
    expo_sector = defaultdict(float)
    total = 0.0
    for pos in open_positions:
        amt = pos['invest']
        tkr = pos['ticker']
        sec = sectors_by_ticker.get(tkr, 'UNKNOWN')
        expo_ticker[tkr] += amt
        expo_sector[sec] += amt
        total += amt
    return total, dict(expo_ticker), dict(expo_sector)

#### Throttle Book When in Drawdown 
Apply multiplicative haircut to utilisation caps based on current peak to trough drawdown 

In [17]:
def utilization_throttle(equity_curve):
    if not equity_curve:
        return 1.0
    peak = max(equity_curve)
    curr = equity_curve[-1]
    dd = 0.0 if peak == 0 else (peak - curr)/peak
    mult = 1.0
    for level, m in DRAWDOWN_THROTTLE_LEVELS:
        if dd >= level:
            mult = min(mult, m)
    return mult

#### Cap Each Proposed Trade
When sizing a new trade, clamp it by all remaining budgets

In [18]:
def clamp_by_caps(proposed_amt, capital, 
                  total_open, new_today_open, expo_ticker, expo_sector,
                  ticker, sector,
                  util_mult=1.0):
    # remaining dollar budgets
    rem_total  = MAX_TOTAL_UTILIZATION*util_mult*capital - total_open
    rem_new    = MAX_NEW_DAILY_UTILIZATION*util_mult*capital - new_today_open
    rem_ticker = MAX_TICKER_UTILIZATION*util_mult*capital - expo_ticker.get(ticker, 0.0)
    rem_sector = MAX_SECTOR_UTILIZATION*util_mult*capital - expo_sector.get(sector, 0.0)
    per_trade  = MAX_POSITION_SIZE*util_mult*capital

    # clamp
    max_affordable = max(0.0, min(proposed_amt, rem_total, rem_new, rem_ticker, rem_sector, per_trade))
    return max_affordable

#### Calculate Historical Payouts
Estimate average upside “b” per winning trade for Kelly sizing. 

Only looks at wins; losses are implicit in Kelly’s (1-p)/b term.


In [19]:
def calculate_historical_payouts(df):
    """
    Returns a dict per ticker with:
      - 'b': avg_win / avg_loss_abs  (Kelly 'odds' ratio)
      - 'avg_win': mean positive next-day return when target==1
      - 'avg_loss_abs': mean absolute negative next-day return when target==0
    """
    out = {}
    for t, g in df.groupby('ticker'):
        # next_day return already implicit in your create_target_variable
        r = (g['next_day_close'] - g['close_price']) / g['close_price']
        wins = r[g['target'] == 1]
        losses = r[g['target'] == 0]
        avg_win = wins[wins > 0].mean() if (wins > 0).any() else np.nan
        avg_loss_abs = (-losses[losses < 0]).mean() if (losses < 0).any() else np.nan

        if np.isfinite(avg_win) and np.isfinite(avg_loss_abs) and avg_loss_abs > 0:
            b = avg_win / avg_loss_abs
        else:
            b = np.nan

        out[t] = {'b': b, 'avg_win': avg_win, 'avg_loss_abs': avg_loss_abs}
    return out


##### PyTorch Helpers

In [20]:
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
    def __len__(self): return self.X.shape[0]
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class LSTMClassifier(nn.Module):
    def __init__(self, n_features, hidden1=128, hidden2=64, fc=32, dropout=0.3, inter_dropout=0.1, use_layernorm=False):
        super().__init__()
        self.lstm1 = nn.LSTM(
            input_size=n_features, hidden_size=hidden1,
            num_layers=1, batch_first=True, bidirectional=False, dropout=0.1
        )
        self.inter_drop = nn.Dropout(p=inter_dropout)  # proxy for recurrent_dropout
        self.lstm2 = nn.LSTM(
            input_size=hidden1, hidden_size=hidden2,
            num_layers=1, batch_first=True, bidirectional=False, dropout=0.1
        )

        # Normalization after temporal pooling
        self.use_layernorm = bool(use_layernorm)
        if self.use_layernorm:
            self.norm = nn.LayerNorm(normalized_shape=hidden2)
        else:
            self.norm = nn.BatchNorm1d(num_features=hidden2)

        # Head
        self.fc1 = nn.Linear(hidden2, fc)
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(p=dropout)
        self.fc_out = nn.Linear(fc, 1)  # logits

    def forward(self, x):
        # x: (B, T, F)
        out1, _ = self.lstm1(x)     # (B, T, hidden1)
        out1 = self.inter_drop(out1)
        out2, _ = self.lstm2(out1)  # (B, T, hidden2)

        last = out2[:, -1, :]       # (B, hidden2)
        if isinstance(self.norm, nn.BatchNorm1d):
            last = self.norm(last)  # BN expects (B, C)
        else:
            last = self.norm(last)  # LN expects (B, C)

        z = self.fc1(last)
        z = self.relu(z)
        z = self.drop(z)
        logits = self.fc_out(z).squeeze(-1)  # (B,)
        return logits

class EarlyStopper:
    def __init__(self, patience=15, mode='min'):
        self.patience = patience
        self.counter = 0
        self.best_metric = None
        self.best_state_dict = None
        self.mode = mode  # 'min' for val_loss
    def step(self, metric, model):
        if self.best_metric is None:
            improved = True
        else:
            improved = (metric < self.best_metric) if self.mode == 'min' else (metric > self.best_metric)
        if improved:
            self.best_metric = metric
            self.best_state_dict = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            self.counter = 0
        else:
            self.counter += 1
        return improved

@torch.no_grad()
def _evaluate(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    all_logits = []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        total_loss += loss.item() * xb.size(0)
        all_logits.append(logits.detach().cpu())
    avg_loss = total_loss / len(loader.dataset)
    logits = torch.cat(all_logits, dim=0).numpy()
    probs = 1.0 / (1.0 + np.exp(-logits))  # sigmoid
    return avg_loss, probs

#### Train Company Models
Train, calibrate, and persist a per-ticker classifier.
This encapsulates per-asset modelling with proper scaling and probability calibration, which is crucial since Kelly needs probabilities (not raw logits).

In [21]:
def train_company_models(company_data_df: pd.DataFrame,
                         ticker: str,
                         feature_cols: list,
                         model_save_path: str,
                         sequence_length: int = None,
                         H = 1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #sequence length & guards
    if sequence_length is None:
        sequence_length = calculate_dynamic_sequence_length(company_data_df)

    if len(company_data_df) < sequence_length + H + 10:
        print(f"Not enough data for {ticker}.")
        return False, sequence_length

    contiguous_periods = identify_contiguous_periods(company_data_df)
    if not contiguous_periods:
        print(f"No contiguous periods found for {ticker}.")
        return False, sequence_length

    #build sequences
    targets = company_data_df[f'target_{H}d'].values
    X, y = create_contiguous_sequences(
        company_data_df[feature_cols].values,
        targets,
        contiguous_periods,
        sequence_length,
        H
    )
    print(f"Created {len(X)} sequences for {ticker}.")
    if len(X) < 2:
        return False, sequence_length

    #chronological split (no shuffle)
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    #Scale features (fit on train, transform both)
    scaler = MinMaxScaler()
    F = X_train.shape[-1]
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, F)).reshape(X_train.shape)
    X_val_scaled   = scaler.transform(  X_val.reshape(-1, F)).reshape(X_val.shape)

    # Keras fit used the scaled data
    train_ds = SequenceDataset(X_train_scaled, y_train)
    val_ds   = SequenceDataset(X_val_scaled,   y_val)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  drop_last=True)
    val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, drop_last=False)

    n_features = X_train.shape[-1]
    model = LSTMClassifier(n_features=n_features, dropout=0.3, inter_dropout=0.1).to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)  # clipnorm via grad clipping below
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=7, min_lr=1e-7)
    early = EarlyStopper(patience=15, mode='min')

    for epoch in range(100):
        model.train()
        total_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # clipnorm=1.0
            optimizer.step()
            total_loss += loss.item() * xb.size(0)

        train_loss = total_loss / len(train_loader.dataset)
        val_loss, _ = _evaluate(model, val_loader, device, loss_fn)

        scheduler.step(val_loss)  # ReduceLROnPlateau on val_loss

        # EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
        _ = early.step(val_loss, model)
        if early.counter >= early.patience:  # stop after 'patience' non-improve epochs
            break

    # Restore best weights
    if early.best_state_dict is not None:
        model.load_state_dict(early.best_state_dict)

    #final validation pass for isotonic calibration (on scaled val data)
    _, validation_predictions = _evaluate(model, val_loader, device, loss_fn)

    # Keras code used 10% buffer (comment said 5%)
    pred_min, pred_max = validation_predictions.min(), validation_predictions.max()
    buffer = 0.05 * (pred_max - pred_min)
    y_min_dynamic = max(0.0, pred_min - buffer)
    y_max_dynamic = min(1.0, pred_max + buffer)

    calibrator = IsotonicRegression(y_min=y_min_dynamic, y_max=y_max_dynamic, out_of_bounds='clip')
    calibrator.fit(validation_predictions, y_val.astype(float))

    # inside train_company_models(..., H=H)
    os.makedirs(model_save_path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(model_save_path, f"{ticker}_lstm_H{H}.pt"))
    joblib.dump(calibrator,      os.path.join(model_save_path, f"{ticker}_calibrator_H{H}.pkl"))
    joblib.dump(scaler,          os.path.join(model_save_path, f"{ticker}_scaler_H{H}.pkl"))
    joblib.dump(sequence_length, os.path.join(model_save_path, f"{ticker}_seq_length_H{H}.pkl"))


    del model, scaler, calibrator, train_loader, val_loader, train_ds, val_ds
    del X_train, X_val, y_train, y_val, X_train_scaled, X_val_scaled
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return True, sequence_length

#### Predict Next Day Performance 
One-step-ahead inference for a ticker on a given day.

Feeds calibrated p(up) into the portfolio/Kelly step.

In [22]:
def predict_next_horizon(company_data_df: pd.DataFrame,
                                 ticker: str,
                                 feature_cols: list,
                                 model_save_path: str,
                                 H: int) -> dict | None:

    try:
        # Load artifacts
        state_dict_path = os.path.join(model_save_path, f"{ticker}_lstm_H{H}.pt")
        calibrator = joblib.load(os.path.join(model_save_path, f"{ticker}_calibrator_H{H}.pkl"))
        scaler     = joblib.load(os.path.join(model_save_path, f"{ticker}_scaler_H{H}.pkl"))
        sequence_length    = joblib.load(os.path.join(model_save_path, f"{ticker}_seq_length_H{H}.pkl"))


        if not os.path.isfile(state_dict_path):
            return None
        
        if hasattr(scaler, "n_features_in_") and scaler.n_features_in_ != len(feature_cols):
            return None

        # Build model skeleton with correct input dim, then load weights
        n_features = len(feature_cols)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = LSTMClassifier(n_features=n_features).to(device)
        model.load_state_dict(torch.load(state_dict_path, map_location=device))
        model.eval()
    except Exception:
        return None

    # Find the last contiguous block and take the most recent sequence_length rows
    contiguous_periods = identify_contiguous_periods(company_data_df)
    if not contiguous_periods:
        return None

    last_start, last_end = contiguous_periods[-1]
    period_data = company_data_df.iloc[last_start:last_end + 1]
    if len(period_data) < sequence_length:
        return None

    last_sequence = period_data.tail(sequence_length)

    # Scale using the saved scaler (fit  train only)
    # Ensure column order matches training
    try:
        scaled_features = scaler.transform(last_sequence[feature_cols])
    except Exception:
        return None

    # Shape: (1, T, F) as float32 tensor
    x = torch.from_numpy(np.asarray([scaled_features], dtype=np.float32)).to(device)

    with torch.no_grad():
        logits = model(x)                  # shape (1,)
        prob = torch.sigmoid(logits).item()  # scalar in [0,1]

    # Isotonic calibration (expects iterable)
    calibrated = float(calibrator.predict([prob])[0])

    print(f"Raw prediction for {ticker}: {prob}")
    print(f"Calibrated prediction for {ticker}: {calibrated}")

    return {
        'ticker': ticker,
        'raw_prediction': prob,
        'calibrated_prediction': calibrated
    }

#### Select and Size Portfolio
Turn a cross-section of calibrated predictions into position sizes.

Enforces diversification (one per sector) and risk discipline (Fractional Kelly).

In [23]:
def select_and_size_portfolio(daily_predictions_df: pd.DataFrame, payout_map: dict,
                            total_capital: float, sector_threshold: float,
                            kelly_fraction: float) -> pd.DataFrame:

    investment_decisions = []

    if daily_predictions_df.empty:
        return pd.DataFrame()

    for sector, group in daily_predictions_df.groupby('sector'):
        avg_sector_score = group['calibrated_prediction'].mean()
        if avg_sector_score < sector_threshold:
            continue

        best_stock_in_sector = group.loc[group['calibrated_prediction'].idxmax()]
        ticker = best_stock_in_sector['ticker']
        p = best_stock_in_sector['calibrated_prediction']
        b_info = payout_map.get(ticker, 0.0)
        b = (b_info['b'] if isinstance(b_info, dict) else float(b_info))
        if not np.isfinite(b) or b <= 0: 
            continue
        
        if not b_info or not np.isfinite(b_info.get('b', np.nan)): 
            continue
        b = max(b_info['b'], 1e-6)  # avoid div-by-zero
        kelly_percentage = p - (1 - p) / b

        if kelly_percentage > 0:
            investment_fraction = kelly_percentage * kelly_fraction
            investment_amount = total_capital * investment_fraction

            print(f"Investing {investment_amount} in {ticker}")

            investment_decisions.append({
                'ticker': ticker,
                'investment_fraction': investment_fraction,
                'investment_amount': investment_amount,
                'predicted_prob': p
            })

    return pd.DataFrame(investment_decisions)

#### Models Exist for Ticker
Quick guard to avoid retraining if a ticker’s artifacts already exist.

Decide whether to train.

In [24]:
def models_exist_for_ticker(ticker: str, model_path: str, H: int = 1) -> bool:
    files = [
        f"{ticker}_lstm_H{H}.pt",
        f"{ticker}_calibrator_H{H}.pkl",
        f"{ticker}_scaler_H{H}.pkl",
        f"{ticker}_seq_length_H{H}.pkl",
    ]
    return all(os.path.exists(os.path.join(model_path, f)) for f in files)

#### Run Simulation
The “engine” — walks forward over dates, trains as needed, predicts, sizes, books PnL, and updates capital.

Enforces chronological integrity, periodic retraining, diversification, and proper capital tracking.

In [25]:
def run_simulation(master_df: pd.DataFrame, payout_map: dict, feature_cols: list,
                   initial_capital: float, initial_training_days: int = INITIAL_TRAINING_DAYS,
                   H: int = 1, allow_overlap: bool = True):

    print(f"\nStarting simulation with {initial_training_days} initial training days... (H={H})")
    capital = initial_capital
    equity_curve = [capital]
    simulation_log = []

    # Precompute horizon returns if not already present
    ret_col = f"ret_{H}d"
    tgt_col = f"target_{H}d"
    if ret_col not in master_df.columns or tgt_col not in master_df.columns:
        print("Adding horizon targets to master_df...")
        master_df = add_horizon_targets(master_df, H=H, price_col='close_price')

    sectors_by_ticker = master_df.groupby('ticker')['sector'].first().to_dict()

    all_tickers = master_df['ticker'].unique()
    retrain_counter = {ticker: RETRAIN_INTERVAL for ticker in all_tickers}
    ticker_sequence_lengths = {}

    unique_dates = sorted(master_df['date'].unique())
    start_index = min(initial_training_days, len(unique_dates) - 1)
    print(f"Starting predictions from day {start_index} (after {initial_training_days} training days)")

    # Overlapping book
    open_positions = []  # each: {'ticker','invest','entry_date','exit_date'}
    prev_equity_end = capital

    for i in tqdm(range(start_index, len(unique_dates)), desc="Simulating Trading Days"):
        current_date = unique_dates[i]

        # settle positions exiting today
        total_pnl_today = 0.0
        still_open = []
        for pos in open_positions:
            if pos['exit_date'] == current_date:
                # realize H-day return measured from entry date
                row = master_df[(master_df['date'] == pos['entry_date']) & (master_df['ticker'] == pos['ticker'])]
                if not row.empty:
                    rH = row.iloc[0][ret_col]
                    if pd.notna(rH):
                        pnl = pos['invest'] * rH
                        total_pnl_today += pnl
                        capital += pos['invest'] + pnl  # release capital
                # else: silently drop if missing
            else:
                still_open.append(pos)
        open_positions = still_open

        # equity at START (after settlements, before new entries)
        open_notional_start = sum(p['invest'] for p in open_positions)
        equity_start = capital + open_notional_start
        daily_return = (equity_start / prev_equity_end) - 1.0 if prev_equity_end > 0 else 0.0

        # build prediction universe using info strictly before current_date
        historical_data = master_df[master_df['date'] < current_date]
        if i == 0:  # no "yesterday" slice for display
            todays_data_for_prediction = pd.DataFrame(columns=master_df.columns)
        else:
            todays_data_for_prediction = master_df[master_df['date'] == unique_dates[i-1]]

        daily_predictions = []
        for ticker in todays_data_for_prediction['ticker'].unique():
            company_hist_data = historical_data[historical_data['ticker'] == ticker]
            if company_hist_data.empty:
                continue

            need_retrain = retrain_counter.get(ticker, 0) >= RETRAIN_INTERVAL
            have_models  = models_exist_for_ticker(ticker, MODEL_SAVE_PATH, H=H)
            if need_retrain or not have_models:
                training_success, seq_length = train_company_models(
                    company_hist_data, ticker, feature_cols, MODEL_SAVE_PATH, H=H
                )
                if training_success:
                    ticker_sequence_lengths[ticker] = seq_length
                    retrain_counter[ticker] = 0
                else:
                    continue

            pred = predict_next_horizon(company_hist_data, ticker, feature_cols, MODEL_SAVE_PATH, H=H)
            if pred:
                info = todays_data_for_prediction[todays_data_for_prediction['ticker'] == ticker].iloc[0]
                pred.update({'company_name': info['company_name'], 'sector': info['sector']})
                daily_predictions.append(pred)
                retrain_counter[ticker] += 1

        daily_predictions_df = pd.DataFrame(daily_predictions)

        # size raw trades via Kelly
        investment_decision_df = select_and_size_portfolio(
            daily_predictions_df, payout_map, capital, SECTOR_CONFIDENCE_THRESHOLD, KELLY_FRACTION
        )

        # entry logic (allow_overlap controls laddering)
        exit_idx = i + H
        if exit_idx < len(unique_dates):
            exit_date = unique_dates[exit_idx]
        else:
            exit_date = None  # can't open new trades that we cannot exit within data

        entries_today = []
        if exit_date is not None and not investment_decision_df.empty:
            # strongest first to grab budgets
            investment_decision_df = investment_decision_df.sort_values('predicted_prob', ascending=False)

            # current exposures (post-settlement)
            util_mult = utilization_throttle(equity_curve)
            total_open, expo_ticker, expo_sector = compute_exposures(open_positions, sectors_by_ticker)
            new_today_open = 0.0

            for _, trade in investment_decision_df.iterrows():
                tkr = trade['ticker']
                sec = sectors_by_ticker.get(tkr, 'UNKNOWN')

                # per-ticker ladder depth
                open_count = sum(1 for p in open_positions if p['ticker'] == tkr)
                if allow_overlap:
                    if open_count >= MAX_POSITIONS_PER_TICKER:
                        continue
                else:
                    if open_count >= 1:
                        continue  # no overlapping allowed

                # base size from Kelly (already scaled by KELLY_FRACTION)
                base_amt = float(trade['investment_amount'])
                base_amt = min(base_amt, KELLY_TRADE_CAP * (capital + sum(p['invest'] for p in open_positions)))  # vs equity

                # clamp by remaining budgets
                amt = clamp_by_caps(base_amt, (capital + sum(p['invest'] for p in open_positions)),
                                    total_open, new_today_open, expo_ticker, expo_sector,
                                    tkr, sec, util_mult)
                if amt < MIN_TRADE_DOLLARS:
                    continue

                # lock capital and record position
                capital -= amt
                total_open += amt
                new_today_open += amt
                expo_ticker[tkr] = expo_ticker.get(tkr, 0.0) + amt
                expo_sector[sec] = expo_sector.get(sec, 0.0) + amt

                pos = {'ticker': tkr, 'invest': amt, 'entry_date': current_date, 'exit_date': exit_date}
                open_positions.append(pos)
                entries_today.append({**trade.to_dict(), 'allocated_amount': amt, 'exit_date': exit_date})

        # equity at END (after entries)
        open_notional_end = sum(p['invest'] for p in open_positions)
        equity_end = capital + open_notional_end

        # log the day
        simulation_log.append({
            'date': current_date,
            'capital_start': equity_start - open_notional_start,  # cash only
            'capital_end': capital,                               # cash only
            'equity_start': equity_start,
            'equity_end': equity_end,
            'daily_pnl_realized': total_pnl_today,                # realized PnL today
            'daily_return': daily_return,
            'investments_made': entries_today
        })

        equity_curve.append(equity_end)
        prev_equity_end = equity_end

    return pd.DataFrame(simulation_log)


#### Calculate Final Results


In [26]:
def calculate_final_results(simulation_log: pd.DataFrame, initial_capital: float, rf_annual: float = 0.0):
    if simulation_log.empty:
        print("Simulation log is empty. No results to calculate.")
        return

    use_equity = {'equity_start', 'equity_end'}.issubset(simulation_log.columns)

    # precomputed daily_return if available and sane
    if 'daily_return' in simulation_log.columns:
        daily_returns = simulation_log['daily_return'].astype(float).values
    elif use_equity:
        eq_start = simulation_log['equity_start'].astype(float).values
        eq_end   = simulation_log['equity_end'].astype(float).values
        # r_t = equity_start_t / equity_end_{t-1} - 1
        prev_end = np.roll(eq_end, 1)
        prev_end[0] = eq_end[0] if eq_end[0] != 0 else eq_start[0]
        daily_returns = (eq_start / prev_end) - 1.0
    else:
        # cash-based (not ideal for H>0)
        print("Warning: Using cash-based returns")
        cap_start = simulation_log['capital_start'].astype(float).values
        cap_end   = simulation_log['capital_end'].astype(float).values
        prev_end  = np.roll(cap_end, 1)
        prev_end[0] = cap_end[0] if cap_end[0] != 0 else cap_start[0]
        daily_returns = (cap_start / prev_end) - 1.0

    daily_returns = np.where(np.isfinite(daily_returns), daily_returns, 0.0)

    # ROI from final equity if present, else cash
    end_col = 'equity_end' if use_equity else 'capital_end'
    final_value = float(simulation_log[end_col].iloc[-1])
    total_roi = (final_value / initial_capital) - 1.0

    # Sharpe
    excess = daily_returns - (rf_annual / 252.0)
    vol = excess.std(ddof=1)
    sharpe_ratio = (excess.mean() / vol * np.sqrt(252.0)) if vol > 1e-12 else 0.0

    print("\n--- Simulation Results ---")
    print(f"Initial Capital: ${initial_capital:,.2f}")
    print(f"Final Capital:   ${final_value:,.2f}")
    print(f"Total Return on Investment (ROI): {total_roi:.2%}")
    print(f"Annualized Sharpe Ratio: {sharpe_ratio:.2f}")
    print("--------------------------")


In [27]:
companies = pd.read_parquet('stocknet-dataset/stock_table.parquet')
tweets = pd.read_parquet('stocknet-dataset/stock_tweets_withsentiment_withemotion_withstance_nomerge.parquet')
stocks = pd.read_parquet('stocknet-dataset/stock_prices.parquet')

companies = companies.rename(columns={'symbol': 'ticker'})

companies.columns = [x.lower() for x in companies.columns]
tweets.columns = [x.lower() for x in tweets.columns]
stocks.columns = [x.lower() for x in stocks.columns]

tweets['stance_positive'] = (tweets['stance_label'] == 'Positive').astype(int)
tweets['stance_negative'] = (tweets['stance_label'] == 'Negative').astype(int)

tweets_merged = tweets.groupby(['date', 'ticker'], as_index=False).agg({
    'text': lambda x: ' '.join(x),
    'sentiment': lambda x: x.mean(),
    'emotion_anger': 'sum',
    'emotion_disgust': 'sum',
    'emotion_fear': 'sum',
    'emotion_joy': 'sum',
    'emotion_neutral': 'sum',
    'emotion_sadness': 'sum',
    'emotion_surprize': 'sum',
    'stance_positive': 'sum',
    'stance_negative': 'sum'
})




tweets_merged['date'] = pd.to_datetime(tweets_merged['date'])
stocks['date'] = pd.to_datetime(stocks['date'])

"""
master_df = stocks.merge(
    tweets_merged,
    on=["date", "ticker"]
)
"""


master_df = pd.merge(
    stocks,
    tweets_merged,
    on=["date", "ticker"],
    how='left'
)

tweet_feature_cols = ['sentiment', 'emotion_anger', 'emotion_disgust', 'emotion_fear', 'emotion_joy', 'emotion_neutral', 'emotion_sadness', 'emotion_surprize', 'stance_positive', 'stance_negative']
for col in tweet_feature_cols:
    if col in master_df.columns:
        master_df[col].fillna(0, inplace=True)



companies = companies.rename(columns={'symbol': 'ticker'})

master_df = pd.merge(master_df, companies[['ticker', 'sector', 'company']], on='ticker', how='left')


feature_cols = ['open','high','low','volume']

master_df = master_df.rename(columns={'close': 'close_price', 'company': 'company_name'})


print(f"Shape of master_df before dropping NaNs: {master_df.shape}")
#master_df.dropna(inplace=True)
print(f"Shape of master_df after dropping NaNs: {master_df.shape}")

master_df.rename(columns={'close_price': 'close'}, inplace=True)





master_df.sort_values(by=['ticker', 'date'], inplace=True)


def apply_ta_indicators(df_group):
    df_group.set_index(pd.DatetimeIndex(df_group['date']), inplace=True)
    #Trend
    df_group.ta.ema(length=12, append=True)
    df_group.ta.ema(length=26, append=True)
    df_group.ta.ema(length=50, append=True)

    df_group.ta.macd(fast=12, slow=26, signal=9, append=True)



    df_group.ta.rsi(length=14, append=True)
    df_group.ta.stochrsi(length=14, append=True)


    df_group.ta.atr(length=14, append=True)

    bb = ta.bbands(df_group['close'], length=20, std=2)
    df_group['BB_upper'] = bb['BBU_20_2.0']
    df_group['BB_middle'] = bb['BBM_20_2.0']
    df_group['BB_lower'] = bb['BBL_20_2.0']


    df_group.ta.obv(append=True)
    return df_group.reset_index(drop=True)

master_df = master_df.groupby('ticker').apply(apply_ta_indicators)


Shape of master_df before dropping NaNs: (108592, 21)
Shape of master_df after dropping NaNs: (108592, 21)


In [28]:
master_df.drop(columns=['text','adj close','sentiment','emotion_anger','emotion_disgust','emotion_fear','emotion_joy','emotion_neutral','emotion_sadness','emotion_surprize'], inplace=True)


In [29]:
master_df


date       open       high        low      close  \
ticker                                                               
AAPL   0    2012-09-04  95.108574  96.448570  94.928574  96.424286   
       1    2012-09-05  96.510002  96.621429  95.657143  95.747147   
       2    2012-09-06  96.167145  96.898575  95.828575  96.610001   
       3    2012-09-07  96.864288  97.497147  96.538574  97.205711   
       4    2012-09-10  97.207146  97.612854  94.585716  94.677139   
...                ...        ...        ...        ...        ...   
XOM    1253 2017-08-28  76.900002  76.940002  76.260002  76.470001   
       1254 2017-08-29  76.209999  76.489998  76.080002  76.449997   
       1255 2017-08-30  76.239998  76.449997  76.059998  76.099998   
       1256 2017-08-31  76.269997  76.489998  76.050003  76.330002   
       1257 2017-09-01  76.370003  76.849998  76.320000  76.570000   

                  volume ticker  stance_positive  stance_negative  \
ticker                                                              
AAPL   0      91973000.0   AAPL              0.0              0.0   
       1      84093800.0   AAPL              0.0              0.0   
       2      97799100.0   AAPL              0.0              0.0   
       3      82416600.0   AAPL              0.0              0.0   
       4     121999500.0   AAPL              0.0              0.0   
...                  ...    ...              ...              ...   
XOM    1253    8229700.0    XOM              0.0              0.0   
       1254    7060400.0    XOM              0.0              0.0   
       1255    8218000.0    XOM              0.0              0.0   
       1256   15641700.0    XOM              0.0              0.0   
       1257    7340800.0    XOM              0.0              0.0   

                       sector  ... MACDh_12_26_9  MACDs_12_26_9     RSI_14  \
ticker                         ...                                           
AAPL   0       Consumer Goods  ...           NaN            NaN        NaN   
       1       Consumer Goods  ...           NaN            NaN        NaN   
       2       Consumer Goods  ...           NaN            NaN        NaN   
       3       Consumer Goods  ...           NaN            NaN        NaN   
       4       Consumer Goods  ...           NaN            NaN        NaN   
...                       ...  ...           ...            ...        ...   
XOM    1253  Basic Matierials  ...     -0.107548      -0.972858  31.975492   
       1254  Basic Matierials  ...     -0.069077      -0.990127  31.851847   
       1255  Basic Matierials  ...     -0.054652      -1.003790  29.688704   
       1256  Basic Matierials  ...     -0.018917      -1.008519  32.913052   
       1257  Basic Matierials  ...      0.028984      -1.001273  36.200731   

             STOCHRSIk_14_14_3_3  STOCHRSId_14_14_3_3   ATRr_14   BB_upper  \
ticker                                                                       
AAPL   0                     NaN                  NaN       NaN        NaN   
       1                     NaN                  NaN       NaN        NaN   
       2                     NaN                  NaN       NaN        NaN   
       3                     NaN                  NaN       NaN        NaN   
       4                     NaN                  NaN       NaN        NaN   
...                          ...                  ...       ...        ...   
XOM    1253            35.117121            31.775404  0.786087  81.525829   
       1254            48.597552            38.712818  0.759224  81.303475   
       1255            55.025431            46.246701  0.732850  80.964170   
       1256            73.940933            59.187972  0.711932  80.569554   
       1257            85.986580            71.650981  0.698937  80.167620   

             BB_middle   BB_lower          OBV  
ticker                                          
AAPL   0           NaN        NaN   91973000.0  
       1           NaN        Na

In [30]:
columns_to_check = ['EMA_12', 'EMA_26','EMA_50','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','RSI_14','ATRr_14','STOCHRSIk_14_14_3_3','STOCHRSId_14_14_3_3','ATRr_14','BB_upper','BB_middle','BB_lower','OBV']
master_df = master_df.dropna(subset=columns_to_check)

In [31]:
feature_cols = ['open','high','low','volume',
                'stance_positive','stance_negative'
                ]

new_indicator_columns = [
    'EMA_12', 'EMA_26', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
    'RSI_14', 'ATRr_14', 'STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3',
    'BB_upper', 'BB_middle', 'BB_lower', 'OBV'
]
feature_cols.extend(new_indicator_columns)
print(f"Final feature columns: {feature_cols}")

Final feature columns: ['open', 'high', 'low', 'volume', 'stance_positive', 'stance_negative', 'EMA_12', 'EMA_26', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_14', 'ATRr_14', 'STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3', 'BB_upper', 'BB_middle', 'BB_lower', 'OBV']


In [32]:
master_df = master_df.rename(columns={'close': 'close_price', 'company': 'company_name'})


In [33]:
master_df.reset_index(drop=True, inplace=True)
master_df

,date,open,high,low,close_price,volume,ticker,stance_positive,stance_negative,sector,...,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,ATRr_14,BB_upper,BB_middle,BB_lower,OBV
0,2012-11-14,77.928574,78.207146,76.597145,76.697144,119292600.0,AAPL,0.0,0.0,Consumer Goods,...,-0.538077,-3.703588,25.771012,21.054629,19.582354,2.377852,94.648550,84.401357,74.154164,-1.014356e+09
1,2012-11-15,76.790001,77.071426,74.660004,75.088570,197477700.0,AAPL,0.0,0.0,Consumer Goods,...,-0.537725,-3.838019,23.573491,15.792949,19.993462,2.380310,93.761634,83.514428,73.267223,-1.211834e+09
2,2012-11-16,75.028572,75.714287,72.250000,75.382858,316723400.0,AAPL,0.0,0.0,Consumer Goods,...,-0.455544,-3.951905,24.836267,12.243346,16.363641,2.459547,92.716200,82.679214,72.642228,-8.951103e+08
3,2012-11-19,77.244286,81.071426,77.125717,80.818573,205829400.0,AAPL,0.0,0.0,Consumer Goods,...,0.002769,-3.951213,43.429047,39.692073,22.576123,2.695187,91.617665,82.201285,72.784906,-6.892809e+08
4,2012-11-20,81.701431,81.707146,79.225716,80.129997,160688500.0,AAPL,0.0,0.0,Consumer Goods,...,0.281964,-3.880722,42.011353,69.373201,40.436207,2.679612,91.027780,81.851785,72.675790,-8.499694e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104215,2017-08-28,76.900002,76.940002,76.260002,76.470001,8229700.0,XOM,0.0,0.0,Basic Matierials,...,-0.107548,-0.972858,31.975492,35.117121,31.775404,0.786087,81.525829,78.243500,74.961171,-2.688251e+08
104216,2017-08-29,76.209999,76.489998,76.080002,76.449997,7060400.0,XOM,0.0,0.0,Basic Matierials,...,-0.069077,-0.990127,31.851847,48.597552,38.712818,0.759224,81.303475,78.057500,74.811525,-2.758855e+08
104217,2017-08-30,76.239998,76.449997,76.059998,76.099998,8218000.0,XOM,0.0,0.0,Basic Matierials,...,-0.054652,-1.003790,29.688704,55.025431,46.246701,0.732850,80.964170,77.832500,74.700830,-2.841035e+08
104218,2017-08-31,76.269997,76.489998,76.050003,76.330002,15641700.0,XOM,0.0,0.0,Basic Matierials,...,-0.018917,-1.008519,32.913052,73.940933,59.187972,0.711932,80.569554,77.624500,74.679446,-2.684618e+08


In [ ]:

master_df_with_target = create_target_variable(master_df)
master_df_with_target = add_horizon_targets(master_df_with_target, H=1)
payout_map = calculate_historical_payouts(master_df_with_target)

initial_capital = 100_000.0
simulation_results = run_simulation(
    master_df_with_target,
    payout_map,
    feature_cols,
    initial_capital,
    initial_training_days=INITIAL_TRAINING_DAYS,
    H=5,
    allow_overlap=True
)

display(simulation_results.head(100))
calculate_final_results(simulation_results, initial_capital)

Creating target variable...
Target variable created.

Starting simulation with 1100 initial training days... (H=3)
Adding horizon targets to master_df...
Starting predictions from day 1100 (after 1100 training days)


Simulating Trading Days:   0%|          | 0/108 [00:00<?, ?it/s]

Created 1086 sequences for AAPL.
Raw prediction for AAPL: 0.47579044103622437
Calibrated prediction for AAPL: 0.5876591801643372
Created 1086 sequences for ABB.
Raw prediction for ABB: 0.43729686737060547
Calibrated prediction for ABB: 0.4909090995788574
Created 1005 sequences for ABBV.
Raw prediction for ABBV: 0.5467233657836914
Calibrated prediction for ABBV: 0.5443037748336792
Created 1086 sequences for AEP.
Raw prediction for AEP: 0.577522337436676
Calibrated prediction for AEP: 0.5
Created 529 sequences for AGFS.
Raw prediction for AGFS: 0.5103181004524231
Calibrated prediction for AGFS: 0.5463917255401611
Created 1086 sequences for AMGN.
Raw prediction for AMGN: 0.519843339920044
Calibrated prediction for AMGN: 0.5384615659713745
Created 1086 sequences for AMZN.
Raw prediction for AMZN: 0.5729195475578308
Calibrated prediction for AMZN: 0.5492957830429077
Created 1086 sequences for BA.
Raw prediction for BA: 0.5025452375411987
Calibrated prediction for BA: 0.5625
Created 573 sequ

Simulating Trading Days:   1%|          | 1/108 [05:19<9:30:30, 319.91s/it]

Raw prediction for XOM: 0.5214451551437378
Calibrated prediction for XOM: 0.5250481963157654
Investing 2093.7026044480785 in BP
Investing 4059.619779992085 in GMRE
Investing 1864.3144533945717 in PM
Investing 2581.3087901702193 in MA
Investing 2174.469376758227 in NVS
Investing 3394.1868571420628 in UTX
Investing 2962.981093934814 in CHTR
Investing 3213.317570724874 in VZ
Investing 1617.0849871297105 in PCG
Raw prediction for AAPL: 0.48210766911506653
Calibrated prediction for AAPL: 0.5876591801643372
Raw prediction for ABB: 0.44171446561813354
Calibrated prediction for ABB: 0.4909090995788574
Raw prediction for ABBV: 0.5425863862037659
Calibrated prediction for ABBV: 0.5443037748336792
Raw prediction for AEP: 0.6333481073379517
Calibrated prediction for AEP: 0.7236841917037964
Raw prediction for AGFS: 0.49922165274620056
Calibrated prediction for AGFS: 0.49279484152793884
Raw prediction for AMGN: 0.5203253030776978
Calibrated prediction for AMGN: 0.5384615659713745
Raw prediction for 

...

Raw prediction for UPS: 0.4706104099750519
Calibrated prediction for UPS: 0.46894270181655884
Raw prediction for UTX: 0.6362468600273132
Calibrated prediction for UTX: 0.6666666865348816
Raw prediction for V: 0.4957983195781708
Calibrated prediction for V: 0.5030637383460999
Raw prediction for VZ: 0.7353808283805847
Calibrated prediction for VZ: 0.6576576828956604
Raw prediction for WFC: 0.5062143206596375
Calibrated prediction for WFC: 0.48051461577415466
Raw prediction for WMT: 0.40102916955947876
Calibrated prediction for WMT: 0.5006946325302124
Raw prediction for XOM: 0.5384101271629333
Calibrated prediction for XOM: 0.5250481963157654
Investing 1801.4012861354342 in BP
Investing 884.949838403472 in GMRE
Investing 1604.0379598186362 in PM
Investing 377.72863596031124 in HSBC
Investing 642.7554993006901 in ABBV
Investing 3231.6138752514503 in GE
Investing 2122.956671506886 in MCD
Investing 2764.70707557403 in VZ
Investing 3716.3584012769743 in AEP
Raw prediction for AAPL: 0.48352986

Simulating Trading Days: 100%|██████████| 108/108 [07:51<00:00,  4.36s/it]

Raw prediction for WMT: 0.4109472930431366
Calibrated prediction for WMT: 0.5006946325302124
Raw prediction for XOM: 0.5402898788452148
Calibrated prediction for XOM: 0.5250481963157654
Investing 2135.324723630451 in BP
Investing 1048.9918507661357 in GMRE
Investing 1901.3764115770541 in PM
Investing 447.7477069640654 in HSBC
Investing 761.9022587968836 in ABBV
Investing 3830.6539793004413 in GE
Investing 2516.486416854954 in MCD
Investing 3277.197267208692 in VZ
Investing 4405.254974111214 in AEP


,date,capital_start,capital_end,equity_start,equity_end,daily_pnl_realized,daily_return,investments_made
0,2017-03-30,100000.000000,76039.014486,100000.000000,100000.000000,0.000000,0.000000,"[{'ticker': 'GMRE', 'investment_fraction': 0.0..."
1,2017-03-31,76039.014486,58998.664065,100000.000000,100000.000000,0.000000,0.000000,"[{'ticker': 'AEP', 'investment_fraction': 0.04..."
2,2017-04-03,58998.664065,45777.057804,100000.000000,100000.000000,0.000000,0.000000,"[{'ticker': 'AEP', 'investment_fraction': 0.04..."
3,2017-04-04,69590.276798,54836.561059,99852.233481,99852.233481,-147.766519,-0.001478,"[{'ticker': 'AEP', 'investment_fraction': 0.04..."
4,2017-04-05,71861.345913,56325.501158,99836.667914,99836.667914,-15.565567,-0.000156,"[{'ticker': 'AEP', 'investment_fraction': 0.04..."
...,...,...,...,...,...,...,...,...
95,2017-08-15,72839.662363,57960.574670,101915.506574,101915.506574,81.419511,0.000800,"[{'ticker': 'AEP', 'investment_fraction': 0.04..."
96,2017-08-16,72554.025141,57301.242615,102027.497952,102027.497952,111.991378,0.001099,"[{'ticker': 'AEP', 'investment_fraction': 0.04..."
97,2017-08-17,71767.870230,56625.418026,101899.740449,101899.740449,-127.757503,-0.001252,"[{'ticker': 'UTX', 'investment_fraction': 0.04..."
98,2017-08-18,71380.492548,56319.773967,101775.727278,101775.727278,-124.013171,-0.001217,"[{'ticker': 'UTX', 'investment_fraction': 0.04..."



--- Simulation Results ---
Initial Capital: $100,000.00
Final Capital:   $101,987.97
Total Return on Investment (ROI): 1.99%
Annualized Sharpe Ratio: 2.50
--------------------------


In [35]:
folder_path = 'trained_models/'
if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
else:
    print(f"Folder not found: {folder_path}")

print(f"Contents of {folder_path} after deletion attempt:")
if os.path.exists(folder_path):
    print(os.listdir(folder_path))
else:
    print("Folder does not exist.")

Contents of trained_models/ after deletion attempt:
[]
